In [1]:
import pandas as pd
import numpy as np

# Renewable technology capacity
- from IRENA (247 countries, 2000-2022)

In [13]:
irena_data = pd.read_csv("../../data/global_IRENA_stat/RECAP_20230920-112608.csv",skiprows = 2,sep=',', encoding='latin-1')
irena_data

,Region/country/area,Technology,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Africa,Total renewable energy,22058.388,22199.217,22469.398,22865.877,22862.042,22980.165,22979.151,23347.134,...,30697.227,32633.419,34953.480,37702.799,43052.833,48386.014,50555.126,53899.493,56122.890,58795.963
1,Africa,Wind,139.050,139.050,144.310,153.790,236.110,240.180,327.905,470.100,...,1741.834,2399.398,3320.268,3831.394,4578.263,5471.553,5528.753,6514.653,7370.153,7686.653
2,Africa,Solar,10.311,13.897,15.673,17.789,18.867,22.119,29.277,38.224,...,680.883,1675.126,2212.258,3426.349,5173.261,8094.772,9488.229,10833.274,11628.328,12499.681
3,Algeria,Total renewable energy,276.600,276.600,276.600,276.600,276.600,276.600,249.600,249.600,...,252.600,266.100,329.096,568.196,661.596,686.396,686.396,666.996,587.296,599.296
4,Algeria,Wind,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,10.200,10.200,10.200,10.200,10.000,10.000,10.000,10.000,10.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,Uruguay,Wind,0.150,0.150,0.150,0.150,0.150,0.150,0.150,0.150,...,59.418,481.268,856.756,1211.456,1513.170,1513.170,1516.490,1516.490,1516.490,1516.490
740,Uruguay,Solar,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,1.589,3.707,64.461,88.950,242.618,248.357,253.388,257.640,266.323,279.180
741,Venezuela (Bolivarian Republic of),Total renewable energy,13216.000,13208.000,12492.000,12492.000,13865.000,14415.810,14598.910,14599.710,...,15021.830,15022.010,15281.010,16972.010,16972.020,16971.010,16950.120,16972.270,16972.530,16972.530
742,Venezuela (Bolivarian Republic of),Wind,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,138.200,138.200,138.200,138.200,138.200,138.200,115.850,138.000,138.000,138.000


In [17]:
country_crosswalk = pd.read_excel("../../data/country_names.xlsx",sheet_name = "crosswalk")

In [21]:
year_list = irena_data.columns[2:]

### Convert data format

In [30]:
tech_list = ["Solar", "Wind"]
for this_tech in tech_list:
    irena_tech = irena_data.query("Technology==@this_tech")
    irena_tech = pd.merge(irena_tech,country_crosswalk[["Country Code","IRENA_Name"]],left_on="Region/country/area",right_on="IRENA_Name")
    irena_tech = irena_tech.drop(columns=["IRENA_Name","Region/country/area"])
    irena_tech_melt = pd.melt(irena_tech, id_vars=['Country Code'], value_vars=year_list, var_name="year", value_name=this_tech+"_GW")
    irena_tech_melt = irena_tech_melt.set_index(["Country Code","year"])
    if this_tech == tech_list[0]:
        irena_tech_combined = irena_tech_melt
    else:
        irena_tech_combined[this_tech+"_GW"] = irena_tech_melt[this_tech+"_GW"]
#irena_tech_combined.to_csv("_all_temporal_RE_capacity.csv")
irena_tech_combined = irena_tech_combined.reset_index()

irena_tech_combined["year"] = irena_tech_combined["year"].astype(int)
irena_tech_combined

,Country Code,year,Solar_GW,Wind_GW
0,DZA,2000,0.000,0.00
1,AGO,2000,0.000,0.00
2,BEN,2000,0.000,0.00
3,BWA,2000,0.000,0.00
4,BFA,2000,0.000,0.00
...,...,...,...,...
4687,PRY,2022,1.166,0.02
4688,PER,2022,332.286,408.99
4689,SUR,2022,11.520,0.00
4690,URY,2022,279.180,1516.49


In [49]:
# assume data in 1990 is zero, then interpolated till 2000
#for this_year in np.arange(10)+1990:
#cntry = irena_tech_combined["Country Code"].unique()[5]
cntry = "CHN"
irena_tech_country = irena_tech_combined[irena_tech_combined["Country Code"]==cntry]
irena_tech_country = irena_tech_country.set_index("year")
new_df = pd.DataFrame(data={"year":np.arange(43)+1980})
new_df = new_df.set_index("year")
new_df["Solar_GW"] = irena_tech_country["Solar_GW"] 
new_df.loc[1980,"Solar_GW"] = 0
new_df["Wind_GW"] = irena_tech_country["Wind_GW"] 
new_df.loc[1980,"Wind_GW"] = 0
new_df = new_df.interpolate(method='linear', axis=0)
new_df

,Solar_GW,Wind_GW
year,,
1980,0.00000,0.000
1981,1.67575,17.050
1982,3.35150,34.100
1983,5.02725,51.150
1984,6.70300,68.200
1985,8.37875,85.250
1986,10.05450,102.300
1987,11.73025,119.350
1988,13.40600,136.400
